In [22]:
from __future__ import print_function
import importlib
resources = importlib.import_module("resources")
import resources.MonteCarlo as MC

import numpy as np
import pandas as pd
import random
my_seed = 2010
random.seed(my_seed)

import matplotlib.pyplot as plt
%matplotlib inline 

from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler


import keras
import tensorflow as tf
from keras.layers import Dense, BatchNormalization, Multiply, Lambda, LeakyReLU
from keras.models import Model
from keras.initializers import RandomNormal, RandomUniform
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.constraints import NonNeg
from keras import backend as K

In [23]:
#Import Data
Data = pd.read_csv('data/preprocessedlosAngParking/part-00000-a8814381-7490-4ac5-8769-145ad7e2824a-c000.csv').sample(frac = 1/5e3)
Data['SN'] = pd.to_numeric(Data['SN'], errors ='coerce')
Data['position'] = pd.to_numeric(Data['position'], errors ='coerce')
Data = Data[['SN','position']]
Data.dropna()
Data = Data[Data['SN'] > 1.3e9]
X = Data.values
X = StandardScaler().fit_transform(X)
db = DBSCAN(eps=0.3, min_samples=10).fit(X)
Data['Labels'] = db.labels_

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:590: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [ ]:
def customLoss(yTrue,yPred):
    #return K.sum(K.square(yTrue-yPred))
    return K.sum(K.min(K.square(yTrue - yPred),axis = 1))\
    +lambd*K.sum(K.square(yTrue-yPred))
def customMetric(yTrue,yPred):
    #return K.sum(K.square(yTrue-yPred))
    return K.max(K.min(K.abs(yTrue -yPred),axis = 1))

batch_size = 256
epochs = 15000
lambd  = 1e-6
n_clusters_ = len(set(db.labels_)) - (1 if -1 in db.labels_ else 0)
no_output_range = [1]
no_input = 1
models = {}
results = {}
for i in no_output_range:
    models[i] = []
    results[i] = 0

for i in range(n_clusters_):
    X_i = Data[Data['Labels']==i]['SN'].values
    Y_i = Data[Data['Labels']==i]['position'].values
    for no_output in no_output_range:
        Y_TRANSFORMED = np.repeat(Y_i.reshape((Y_i.shape[0],1)),no_output,axis=1)
        _input_= keras.layers.Input(shape=(no_input,))
    
        neg_pos_mask = K.tf.constant(np.random.choice( [-1,1],[1,no_output]),dtype = tf.float32)
        mask = Lambda( lambda x : neg_pos_mask)(_input_) # _input_ is just added to get this to compile

        l1 = Dense(64, activation='linear', name="hidden_1_dense64",
                        kernel_initializer= RandomUniform(minval=1, maxval=2),
                        bias_initializer=RandomUniform(), kernel_constraint=NonNeg(),                   
                        input_shape=[no_input])(_input_)
        l1a = LeakyReLU(alpha=0.1,name="hidden_1_leakyRelu")(l1)
        l1b = BatchNormalization(gamma_constraint=NonNeg(), name='batch_norm_1')(l1a)

        l2 = Dense(32, activation='linear', name="hidden_2_dense64",
                        kernel_initializer=RandomUniform(minval=1, maxval=2),
                        bias_initializer=RandomUniform(), kernel_constraint=NonNeg())(l1b)
        l2a = LeakyReLU(alpha=0.1,name="hidden_2_leakyRelu")(l2)
        l2a = l2
        l2b = BatchNormalization(gamma_constraint=NonNeg(), name='batch_norm_2')(l2a)

        l3 = Dense(32, activation='relu', name="hidden_3_dense64",
                        kernel_initializer=RandomUniform(minval=.05, maxval=.15),
                        bias_initializer=RandomUniform(), kernel_constraint=NonNeg())(l2b)
        l3a = LeakyReLU(alpha=0.1,name="hidden_3_leakyRelu")(l2)
        l3a = l3
        l3b = BatchNormalization(gamma_constraint=NonNeg(), name='batch_norm_3')(l3a)

        l4 = Dense(no_output, activation='linear', name="hidden_4_dense64",
                        kernel_initializer=RandomUniform(minval=.05, maxval=1),
                        bias_initializer= RandomNormal(), kernel_constraint=NonNeg())(l3b)

        #_output_ =l3
        _output_ = Multiply()([l4,mask])

        model = keras.models.Model(inputs=_input_ , outputs=_output_)

        model.compile(loss=customLoss, metrics=[customMetric],
                  optimizer=keras.optimizers.Nadam(lr=.01, beta_1=0.9, beta_2=0.999, clipvalue=0.5, schedule_decay=0.0001))

        filepath = "results\parking_weights_cluster{}_output{}.hdf5".format(i,no_output)
        checkpoint = ModelCheckpoint(filepath, monitor="val_customMetric", verbose=0, mode = 'min', save_best_only=True)
        model.fit(X_i, Y_TRANSFORMED,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(X_i, Y_TRANSFORMED),
              callbacks=[checkpoint],
              verbose=0)
        model.load_weights(filepath)
        score = model.evaluate(X_i, Y_TRANSFORMED, verbose=0)
        print(score)
        YPred= model.predict(X_i)
        maxError = np.max(np.min(np.abs(YPred - Y_TRANSFORMED),axis=1),axis=0)
        results[no_output] = max(results[no_output],maxError)
        models[no_output].append(model)
print(results)

In [ ]:
X_space = np.linspace(42.4e8, 43.44e8, 400)
X = Data['SN'].values
Y = Data['position'].values
for no_output in no_output_range:
    plt.clf()
    plt.ylim(0,1e7)
    plt.scatter(X, Y,s = .2, c='blue',marker= '^',zorder=2)
    for i in range(n_clusters_):
        filepath = "results\parking_weights_cluster{}_output{}.hdf5".format(i,no_output)        
        _input_= keras.layers.Input(shape=(no_input,))
        neg_pos_mask = K.tf.constant(np.random.choice( [-1,1],[1,no_output]),dtype = tf.float32)
        mask = Lambda( lambda x : neg_pos_mask)(_input_) # _input_ is just added to get this to compile

        l1 = Dense(64, activation='linear', name="hidden_1_dense64",
                        kernel_initializer= RandomUniform(minval=1, maxval=2),
                        bias_initializer=RandomUniform(), kernel_constraint=NonNeg(),                   
                        input_shape=[no_input])(_input_)
        l1a = LeakyReLU(alpha=0.1,name="hidden_1_leakyRelu")(l1)
        l1b = BatchNormalization(gamma_constraint=NonNeg(), name='batch_norm_1')(l1a)

        l2 = Dense(32, activation='linear', name="hidden_2_dense64",
                        kernel_initializer=RandomUniform(minval=1, maxval=2),
                        bias_initializer=RandomUniform(), kernel_constraint=NonNeg())(l1b)
        l2a = LeakyReLU(alpha=0.1,name="hidden_2_leakyRelu")(l2)
        l2a = l2
        l2b = BatchNormalization(gamma_constraint=NonNeg(), name='batch_norm_2')(l2a)

        l3 = Dense(32, activation='relu', name="hidden_3_dense64",
                        kernel_initializer=RandomUniform(minval=.05, maxval=.15),
                        bias_initializer=RandomUniform(), kernel_constraint=NonNeg())(l2b)
        l3a = LeakyReLU(alpha=0.1,name="hidden_3_leakyRelu")(l2)
        l3a = l3
        l3b = BatchNormalization(gamma_constraint=NonNeg(), name='batch_norm_3')(l3a)

        l4 = Dense(no_output, activation='linear', name="hidden_4_dense64",
                        kernel_initializer=RandomUniform(minval=.05, maxval=1),
                        bias_initializer= RandomNormal(), kernel_constraint=NonNeg())(l3b)
        _output_ = Multiply()([l4,mask])
        model = keras.models.Model(inputs=_input_ , outputs=_output_)
        model.load_weights(filepath)
        model_output_linespace = zip(*model.predict(X_space))
        for Y_pred in model_output_linespace:
            plt.plot(X_space,Y_pred, 'k',linewidth = 1, alpha = .6,zorder=1)
    plt.savefig("results/Test_Parking_Cluster_{number}.jpg".format(number = no_output))


